In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [3]:
!pip install lightgbm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.2 MB/s eta 0:00:0000:0100:01


In [7]:
!pip install category_encoders


  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 2.2 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2


In [11]:
pip install --upgrade scikit-learn category_encoders


Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install scikit-learn==1.3.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 1.7 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      Partial import of sklearn during the build process.
      xcrun: error: invalid active developer path (/Library/Developer/CommandLineTools), missing xcrun at: /Library/Developer/CommandLineTools/usr/bin/xcrun
      Traceback (most recent call last):
        File "/private/var/folders/r8/hsg7qj6j02x97jd7mlfqxqlw0000gn/T/pip-build-env-7_0eyd_3/overlay/lib/python3.12/site-packages/setuptools/_distutils/spawn.py", line 70, in spawn
          subprocess.check_call(cmd, env=_inject_macos_ver(env))
        File "/opt/anaconda3/lib/python3.12/subprocess.py", line 413, in check_call
          raise CalledProcessError(r

In [21]:
def load_data():
    train = pd.read_csv("train3.csv")
    test = pd.read_csv("test3.csv")
    target_col = "target" if "target" in train.columns else train.columns[-1]
    return train, test, target_col
train, test, target_col = load_data()

In [23]:
categorical_cols = train.select_dtypes(include=["object", "category"]).columns.tolist()
numerical_cols = train.select_dtypes(include=["int64", "float64"]).columns.tolist()
numerical_cols.remove(target_col)

In [25]:
imputer = SimpleImputer(strategy="most_frequent")
train[categorical_cols] = imputer.fit_transform(train[categorical_cols])
test[categorical_cols] = imputer.transform(test[categorical_cols])

In [29]:



from category_encoders import TargetEncoder

# Now use TargetEncoder directly
encoder = TargetEncoder(cols=categorical_cols)

train[categorical_cols] = encoder.fit_transform(train[categorical_cols], train[target_col])
test[categorical_cols] = encoder.transform(test[categorical_cols])


ImportError: cannot import name 'Tags' from 'sklearn.utils' (/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/__init__.py)

In [33]:
!pip install scikit-learn==1.3.0



  Using cached scikit-learn-1.3.0.tar.gz (7.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      Partial import of sklearn during the build process.
      xcrun: error: invalid active developer path (/Library/Developer/CommandLineTools), missing xcrun at: /Library/Developer/CommandLineTools/usr/bin/xcrun
      Traceback (most recent call last):
        File "/private/var/folders/r8/hsg7qj6j02x97jd7mlfqxqlw0000gn/T/pip-build-env-_pa054e_/overlay/lib/python3.12/site-packages/setuptools/_distutils/spawn.py", line 70, in spawn
          subprocess.check_call(cmd, env=_inject_macos_ver(env))
        File "/opt/anaconda3/lib/python3.12/subprocess.py", line 413, in check_call
          raise CalledProcessError(retcode, cmd)
      subprocess.CalledProc

In [35]:
scaler = StandardScaler()
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

In [37]:
X = train.drop(columns=[target_col])
y = train[target_col]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [39]:

def evaluate_model(model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    print(f"Model: {model.__class__.__name__} | Accuracy: {score:.4f}")
    return score

In [41]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(eval_metric="logloss")
lgbm = LGBMClassifier()

evaluate_model(rf, X_train, X_val, y_train, y_val)
evaluate_model(xgb, X_train, X_val, y_train, y_val)
evaluate_model(lgbm, X_train, X_val, y_train, y_val)

ValueError: could not convert string to float: 'T'

In [ ]:
param_grid_xgb = {
    "max_depth": [3, 5],
    "learning_rate": [0.01, 0.1],
    "n_estimators": [100, 200],
}

grid_xgb = GridSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    param_grid_xgb,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)
grid_xgb.fit(X_train, y_train)
print(f"Best XGBoost params: {grid_xgb.best_params_}")

In [ ]:
param_grid_lgbm = {
    "num_leaves": [20, 31],
    "learning_rate": [0.01, 0.05],
    "n_estimators": [100, 300],
}

random_lgbm = RandomizedSearchCV(
    LGBMClassifier(),
    param_grid_lgbm,
    n_iter=5,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)
random_lgbm.fit(X_train, y_train)
print(f"Best LightGBM params: {random_lgbm.best_params_}")

In [ ]:
final_model = LGBMClassifier(**random_lgbm.best_params_)
final_model.fit(X_train, y_train)
y_final_pred = final_model.predict(X_val)

final_acc = accuracy_score(y_val, y_final_pred)
print(f"Final Model Accuracy: {final_acc:.4f}")

In [49]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [ ]:
gemstone = pd.read_csv("GEMSTONETR.csv")

X = gemstone.drop(columns=["price"])
y = gemstone["price"]